## Python AI Chatbot
This is a chatbot running in python 3.6 virutal environment.  The initial step will be to load that virtual environment. The code for this is provided by https://www.techwithtim.net/tutorials/ai-chatbot/chat-bot-part-2

In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
import sys
print(sys.version)

3.6.13 |Anaconda, Inc.| (default, Feb 23 2021, 12:58:59) 
[GCC Clang 10.0.0 ]


In [2]:
pip install virtualenv

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install tflearn

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install tensorflow

     |████████████████████████████████| 198.9 MB 35.0 MB/s eta 0:00:01
     |████████████████████████████████| 5.6 MB 29.6 MB/s eta 0:00:01
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached flatbuffers-1.12-py2.py3-none-any.whl (15 kB)
     |████████████████████████████████| 2.9 MB 10.1 MB/s eta 0:00:01
     |████████████████████████████████| 132 kB 51.9 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 27.7 MB/s eta 0:00:01
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
     |████████████████████████████████| 979 kB 49.9 MB/s eta 0:00:01
     |████████████████████████████████| 4.4 MB 4.8 MB/s eta 0:00:01
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 1.3 MB 47.4 MB/s eta 0:00:01
  Using cached tensorboard_data_server-0

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os

# current working directory
print(os.getcwd())

/Users/jennyskytta


In [5]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy
import tflearn
import tensorflow
import random
import json
with open('/Users/jennyskytta/Downloads/json/intents.json') as file:
    data = json.load(file)

Instructions for updating:
non-resource variables are not supported in the long term
Scipy not supported!


In [6]:
words = []
labels = []
docs_x = []
docs_y = []

In [7]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])

## Preprocessing 
We are now going to preprocess our data.  We loaded in the JSON file.  We are creating a new list.  For each pattern, we want to add a new element for what intent "tag" its apart of for this chatbot.  So for each entry in docs_x corresponds to each entry in docs_y and the intent will be saved there.  This will be important for training our model.  

## Stemming 
we want to stem all of our words in the words list and remove the duplicates to know the vocabulary size.  We will convert to lowercase and remove those dupes using 'Set'. We then sort everything. Now we want to setup our training and testing output.  

## Bag of Words 
Neural networks only understand numbers rather than strings so we'll use "bag of words" or "onehotencoded" and this will represent the length encoding of the entries for the positions in the list.

In [8]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]  # convert to lowercase 
words = sorted(list(set(words)))  # removes duplicates - takes all words - list converts back into list and then sort

labels = sorted(labels) 

In [9]:
training = []   # Training is a blank list 
output = []   # output is a blank list 

out_empty = [0 for _ in range(len(labels))] 

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [10]:
training = numpy.array(training)
output = numpy.array(output)

## The AI Model

The model has the input layer that is looking at the number of words, next it goes into our two layers with 8 nodes.  Each word connects to a node in the hidden layers.  Finally, Softmax activiation calculates the probabilities for tags and predicting where to draw the response.  Finally, DNN is a type of neural network. The number of epochs is the number of times we're showing the model this data.  

In [15]:
tensorflow.compat.v1.reset_default_graph()  #updated from training as previous was deprecated 

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)  # 8 neurons for the hidden layer
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")  #output layer
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=1000, batch_size=9, show_metric=True)
model.save("model.tflearn")

Training Step: 2999  | total loss: 0.26309 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.26309 - acc: 0.9759 -- iter: 18/26
Training Step: 3000  | total loss: 0.24500 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.24500 - acc: 0.9783 -- iter: 26/26
--
INFO:tensorflow:/Users/jennyskytta/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


## Final Script

here is the final full script 

In [ ]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy
import tflearn
import tensorflow
import random
import json
import pickle

with open('/Users/jennyskytta/Downloads/json/intents.json') as file:
    data = json.load(file)

try:
    with open("data.pickle", "rb") as f:    #rb = read bytes 
        words, labels, training, output = pickle.load(f)
except:
    words = []   # these variables will be saved in pickle file 
    labels = []
    docs_x = []
    docs_y = []

    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])

        if intent["tag"] not in labels:
            labels.append(intent["tag"])

    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))

    labels = sorted(labels)

    training = []
    output = []

    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)


    training = numpy.array(training)
    output = numpy.array(output)

    with open("data.pickle", "wb") as f:
        pickle.dump((words, labels, training, output), f)

tensorflow.compat.v1.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
    model.save("model.tflearn")
    

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return numpy.array(bag)


def chat():
    print("Chat with Skyttabot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        results = model.predict([bag_of_words(inp, words)])[0]
        results_index = numpy.argmax(results)
        tag = labels[results_index]
        
        if results[results_index] > 0.7:
            for tg in data["intents"]:
                if tg['tag'] == tag:
                    responses = tg['responses']

            print(random.choice(responses))
        else:
            print("I'm sorry, what on earth are you trying to convey - be clear- I'm a bot!")

chat()

INFO:tensorflow:Restoring parameters from /Users/jennyskytta/model.tflearn
Chat with Skyttabot (type quit to stop)!
You: test
I'm sorry, what on earth are you trying to convey - be clear- I'm a bot!
